In [1]:
import xml.etree.ElementTree as ET

tree=ET.parse('/content/drive/MyDrive/pubmed20n1001.xml')

In [60]:
root=tree.getroot()

Abstract=root.findall('./PubmedArticle/MedlineCitation/Article/Abstract')

abstractList=[]
for abstract in Abstract:
  abstractText=abstract.find('AbstractText').text
  if type(abstractText)==str:
    abstractList.append(abstractText.lower())

In [9]:
#abstractText tokenize
from tensorflow.keras.preprocessing.text import text_to_word_sequence

word_List=[]
for sentence in abstractList:
  words=text_to_word_sequence(sentence)
  word_List.append(words)

In [62]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

word_List=[]
for sentence in abstractList:
  words=word_tokenize(sentence)
  word_List.append(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
#불용어 제거,특수문자 제거
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words=set(stopwords.words('english'))
remove_set=['.',',',';','(',')',':']
result=[]
for sentence in word_List:
  remove_stopword=[]
  sentence=[i for i in sentence if i not in remove_set]
  for word in sentence:
    if word not in stop_words:
      remove_stopword.append(word)
  result.append(remove_stopword)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
from gensim.models.word2vec import Word2Vec
model=Word2Vec(result,sg=1)

In [70]:
import pandas as pd
print('TNF-α과 가장 유사한 단어와 유사도')
df=pd.DataFrame(model.wv.most_similar('tnf-α'),columns=['단어','유사도'])
df

TNF-α과 가장 유사한 단어와 유사도


,단어,유사도
0,il-10,0.943990
1,il-1β,0.923644
2,il-6,0.917089
3,il-8,0.909551
4,tnfα,0.909075
5,il-4,0.906650
6,il-17,0.900196
7,interleukin-10,0.890973
8,interleukin-6,0.882496
9,interleukin-1β,0.879820


In [66]:
len(model.wv.vocab)

27318

In [67]:
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:30]

['study',
 'patients',
 '%',
 'cells',
 'using',
 'treatment',
 'results',
 'data',
 'used',
 'however',
 'cell',
 'cancer',
 'may',
 'health',
 'also',
 'clinical',
 'disease',
 'studies',
 'associated',
 'analysis',
 'effects',
 'expression',
 'different',
 'model',
 'use',
 'high',
 'risk',
 'two',
 'potential',
 'present']

In [77]:
from math import log
print(vocab.keys())

dict_keys(['objectives', 'article', 'review', 'mechanism', 'clinical', 'significance', 'interaction', 'determine', 'combination', 'highest', 'risk', 'identify', 'key', 'patients', "'", 'factors', 'relation', 'development', 'muscle', 'toxicity', 'literature', 'conducted', 'pubmed', 'embase', 'december', '2018', 'using', 'combined', 'terms', 'statins', '-', 'group', 'individual', 'agents', 'macrolides', 'drug', 'rhabdomyolysis', 'cyp3a4', 'inhibitors', 'forward', 'backward', 'citation', 'tracking', 'relevant', 'english', 'language', 'vivo', 'studies', 'healthy', 'volunteers', 'case', 'reports', 'population', 'included', 'depends', 'type', 'statin', 'macrolide', 'used', 'due', 'inhibition', 'transporter', 'causing', 'increased', 'exposure', 'correlation', "'s", 'could', 'established', 'unless', 'patient', 'advanced', 'age', 'cardiovascular', 'diseases', 'renal', 'impairment', 'diabetes', 'concomitant', 'use', 'simvastatin', 'lesser', 'extent', 'atorvastatin', 'affected', 'rosuvastatin', '